In [1]:
import pandas as pd
import urllib.request
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from haversine import haversine, Unit

## 1) Extract FIgLib Data

In [2]:
# 1) Loop through each directory in http://hpwren.ucsd.edu/HPWREN-FIgLib/HPWREN-FIgLib-Data/
#    Each folder represents a fire event. Grab the entire name
base_url = "http://hpwren.ucsd.edu/HPWREN-FIgLib/HPWREN-FIgLib-Data/"
soup = BeautifulSoup(requests.get(base_url).text)

events = []

for a in soup.table.find_all('a'):
    events.append(a['href'])

#skips the first 4 links in the page (they are the table headers and the parent directory)
events = events[5:]

In [3]:
#    
# 2) Grab the full name of the +00000.jpg file
# This won't get all of the pics, as of 04/07/2022 9 events don't have the +00000.jpg suffix
#
pics = []
not_found = []
for idx,e in enumerate(events):
    e_url = f"{base_url}{e}"
    soup = BeautifulSoup(requests.get(e_url).text)
    found = False
    for a in soup.table.find_all('a'):
        a_href = a['href']
        
        if "00000.jpg" in a_href:
            pics.append((idx, a_href))
            found = True

    if found == False:
        not_found.append(e)

In [4]:
event_df = pd.DataFrame(events, columns=["event_name"])
event_df

,event_name
0,20160604_FIRE_rm-n-mobo-c/
1,20160604_FIRE_smer-tcs3-mobo-c/
2,20160619_FIRE_lp-e-iqeye/
3,20160619_FIRE_om-e-mobo-c/
4,20160619_FIRE_pi-s-mobo-c/
...,...
311,20210209_FIRE_tp-w-mobo-c/
312,20210302_FIRE_lp-e-mobo-c/
313,20210302_FIRE_lp-e-mobo-m/
314,20210319_FIRE_om-n-mobo-c/


In [5]:
pic_df = pd.DataFrame(pics, columns=["idx", "pic_name"])
pic_df = pic_df.set_index("idx")
pic_df

,pic_name
idx,
0,1465065600_+00000.jpg
1,1465065668_+00000.jpg
2,1466359453_+00000.jpg
3,1466359357_+00000.jpg
4,1466359567_+00000.jpg
...,...
311,1612894635_+00000.jpg
312,1614679618_+00000.jpg
313,1614679559_+00000.jpg


In [6]:
event_pic_df = event_df.merge(pic_df, left_index=True, right_index=True, how="inner")
event_pic_df["date"] = event_pic_df["event_name"].str[:8]
event_pic_df["camera_name"] = event_pic_df["event_name"].str.split("_").str[-1].str[:-1]

#Edge case: 2 camera names not following naming convention. Uses "-" instead of "_"
event_pic_df.loc[event_pic_df["camera_name"] == "FIRE-pi-s-mobo-c", ["camera_name"]] = "pi-s-mobo-c"
event_pic_df.loc[event_pic_df["camera_name"] == "FIRE-smer-tcs8-mobo-c", ["camera_name"]] = "smer-tcs8-mobo-c"

event_pic_df["time"] = event_pic_df["pic_name"].str.split("_").str[0]
event_pic_df["datetime"] = pd.to_datetime(event_pic_df["time"], unit="s", origin="unix", utc=True)
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00
...,...,...,...,...,...,...
311,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00
312,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00
313,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00
314,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00


In [7]:
event_pic_df["datetime"].min()

Timestamp('2016-06-04 18:40:00+0000', tz='UTC')

## 2) Camera Metadata Join

In [8]:
camera_metadata_df = pd.read_csv("camera_metadata - camera_metadata.csv")
camera_metadata_df

,camera_id,lat,lat_decimals,long,camera_name,hpweren_camera_description,lat_adjusted,long_adjusted,sample_image_url,camera_model,comment
0,hpwren0_unknown direction,32.550000,2,-117.120000,tje-1-mobo-c,NaN,NaN,NaN,NaN,NaN,NaN
1,hpwren1_north,33.159900,4,-116.808100,bm-n-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
2,hpwren1_east,33.159900,4,-116.808100,bm-e-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
3,hpwren1_south,33.159900,4,-116.808100,bm-s-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
4,hpwren1_west,33.159900,4,-116.808100,bm-w-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1158,Axis-SYP,34.527107,6,-119.978905,syp-n-mobo-c,Santa Ynez Peak,NaN,NaN,NaN,NaN,NaN
1159,Axis-SYP,34.527107,6,-119.978905,syp-w-mobo-c,Santa Ynez Peak,NaN,NaN,NaN,NaN,NaN
1160,hpwren19_east,33.008700,4,-116.970900,wc-e-mobo-c,Mt. Woodson,NaN,NaN,NaN,NaN,NaN
1161,hpwren19_north,33.008700,4,-116.970900,wc-n-mobo-c,Mt. Woodson,NaN,NaN,NaN,NaN,NaN


In [9]:
camera_metadata_filtered_df = camera_metadata_df[["camera_name", "lat", "long", "lat_adjusted", "long_adjusted"]]
event_pic_df = event_pic_df.merge(camera_metadata_filtered_df, left_on="camera_name", right_on="camera_name", how="left")
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00,33.4008,-117.1905,NaN,NaN
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00,NaN,NaN,NaN,NaN
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00,32.7015,-116.7646,NaN,NaN
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00,32.5948,-116.8447,NaN,NaN
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00,32.7300,-116.5800,32.734673,-116.577751
...,...,...,...,...,...,...,...,...,...,...
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,NaN,NaN
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,NaN,NaN
304,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00,NaN,NaN,NaN,NaN
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,NaN,NaN


In [10]:
event_pic_df.loc[event_pic_df["lat_adjusted"].isna(), ["lat_adjusted"]] = event_pic_df["lat"]
event_pic_df.loc[event_pic_df["long_adjusted"].isna(), ["long_adjusted"]] = event_pic_df["long"]

In [11]:
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00,33.4008,-117.1905,33.400800,-117.190500
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00,NaN,NaN,NaN,NaN
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00,32.7015,-116.7646,32.701500,-116.764600
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00,32.5948,-116.8447,32.594800,-116.844700
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00,32.7300,-116.5800,32.734673,-116.577751
...,...,...,...,...,...,...,...,...,...,...
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,33.523500,-116.425700
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,32.701500,-116.764600
304,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00,NaN,NaN,NaN,NaN
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,32.594800,-116.844700


In [12]:
event_pic_df.to_csv("event_pic.csv", index=False)

In [13]:
event_pic_df["datetime"].min()

Timestamp('2016-06-04 18:40:00+0000', tz='UTC')

In [14]:
list(event_pic_df[event_pic_df["lat"].isna()]["camera_name"].unique())

['smer-tcs3-mobo-c',
 'smer-tcs8-mobo-c',
 'smer-tcs9-mobo-c',
 'smer-tcs10-mobo-c',
 'sp-s-mobo-m',
 'om-s-mobo-m',
 'lp-s-mobo-m',
 'om-e-mobo-m',
 'lp-w-mobo-m',
 'om-n-mobo-m',
 'sm-e-mobo-m',
 'wilson-e-mobo-c',
 'lp-e-mobo-m']

## 3) WFABBA Join

In [15]:
time_filter = event_pic_df["datetime"] >= "2021-01-20"
lat_filter = ~event_pic_df["lat"].isna()
overlap_events_df = event_pic_df[time_filter & lat_filter]
overlap_events_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted
300,20210204_FIRE_tp-s-mobo-c/,1612465753_+00000.jpg,20210204,tp-s-mobo-c,1612465753,2021-02-04 19:09:13+00:00,33.5235,-116.4257,33.523500,-116.42570
301,20210209_FIRE_hp-e-mobo-c/,1612892967_+00000.jpg,20210209,hp-e-mobo-c,1612892967,2021-02-09 17:49:27+00:00,33.3600,-116.8400,33.363018,-116.83622
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,33.523500,-116.42570
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,32.701500,-116.76460
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,32.594800,-116.84470
306,20210711_FIRE_wc-e-mobo-c/,1626026154_+00000.jpg,20210711,wc-e-mobo-c,1626026154,2021-07-11 17:55:54+00:00,33.0087,-116.9709,33.008700,-116.97090


In [16]:
# pd.to_datetime(event_pic_df["time"], unit="s", origin="unix")

In [17]:
# GOES 16 operations
wfabba_goes_16_df = pd.read_csv("goes-16.csv")
wfabba_goes_16_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_16_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_16_df["timestamp_converted"]
wfabba_goes_16_df

,Unnamed: 0,Algorithm,Version,Timestamp,Satellite,Instrument,FlightModel,ScanMode,ProductType,FileName,...,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,FD_X,FD_Y,timestamp_converted
0,0,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-18 22:01:19,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,...,301.904,320.516,299.474,33.296,55.546,-9999,51,1137,1140,2021-07-18 22:01:19+00:00
1,1,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-18 22:01:19,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,...,300.849,320.484,299.520,33.294,55.493,-9999,51,1136,1143,2021-07-18 22:01:19+00:00
2,2,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-18 22:01:19,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,...,300.878,320.484,299.520,33.321,55.467,-9999,51,1137,1143,2021-07-18 22:01:19+00:00
3,3,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-18 17:41:19,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021199174119.rev.6_6_001g.FDCC.GOES-16.txt,...,301.791,313.196,297.777,31.178,58.662,-9999,46,1052,1112,2021-07-18 17:41:19+00:00
4,4,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-18 03:46:19,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021199034619.rev.6_6_001g.FDCC.GOES-16.txt,...,292.463,291.827,289.549,100.889,57.824,-9999,51,1140,1055,2021-07-18 03:46:19+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179499,179499,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-18 21:36:02,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021049213602.rev.6_6_001g.FDCC.GOES-16.txt,...,298.127,299.334,296.677,50.981,57.918,-9999,37,1097,1093,2021-02-18 21:36:02+00:00
179500,179500,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-18 11:10:06,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021049111006.rev.6_6_001g.FDCF.GOES-16.txt,...,280.491,277.945,278.480,132.527,60.152,-9999,40,1051,1062,2021-02-18 11:10:06+00:00
179501,179501,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-18 11:16:02,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021049111602.rev.6_6_001g.FDCC.GOES-16.txt,...,280.559,277.939,278.491,131.330,60.152,-9999,40,1051,1062,2021-02-18 11:16:02+00:00
179502,179502,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-18 17:10:05,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021049171005.rev.6_6_001g.FDCF.GOES-16.txt,...,286.542,297.936,287.448,61.695,59.632,-9999,8,1121,1012,2021-02-18 17:10:05+00:00


In [18]:
# GOES 17 operations
wfabba_goes_17_1_df = pd.read_csv("goes-17_1.csv")
# wfabba_goes_17_1_df
wfabba_goes_17_2_df = pd.read_csv("goes-17_2.csv")
# wfabba_goes_17_2_df
wfabba_goes_17_df = pd.concat([wfabba_goes_17_1_df, wfabba_goes_17_2_df])
wfabba_goes_17_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_17_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_17_df

,Unnamed: 0,Algorithm,Version,Timestamp,Satellite,Instrument,FlightModel,ScanMode,ProductType,FileName,...,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,FD_X,FD_Y,timestamp_converted
0,0,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-01-20 22:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021020224031.rev.6_6_001g.FDCF.GOES-17.txt,...,271.089,272.170,270.592,80.607,61.026,-9999,22,3856,656,2021-01-20 22:40:31+00:00
1,1,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-01-20 22:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021020224031.rev.6_6_001g.FDCF.GOES-17.txt,...,276.174,281.399,276.896,70.124,50.907,-9999,22,3333,724,2021-01-20 22:40:31+00:00
2,2,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-01-20 22:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021020224031.rev.6_6_001g.FDCF.GOES-17.txt,...,261.735,274.917,271.265,77.489,57.583,-9999,52,3813,728,2021-01-20 22:40:31+00:00
3,3,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-01-20 22:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021020224031.rev.6_6_001g.FDCF.GOES-17.txt,...,278.536,283.165,277.769,71.630,51.672,-9999,40,3552,781,2021-01-20 22:40:31+00:00
4,4,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-01-20 22:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021020224031.rev.6_6_001g.FDCF.GOES-17.txt,...,293.569,297.363,293.926,68.191,48.742,-9999,24,3303,784,2021-01-20 22:40:31+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509989,509989,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:40:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041004031.rev.6_6_001g.FDCF.GOES-17.txt,...,290.239,291.898,291.449,81.850,43.385,-9999,22,3661,1134,2022-02-10 00:40:31+00:00
509990,509990,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 04:36:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041043617.rev.6_6_001g.FDCC.GOES-17.txt,...,284.100,281.799,282.391,131.324,43.385,-9999,22,3661,1134,2022-02-10 04:36:17+00:00
509991,509991,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:20:31,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041002031.rev.6_6_001g.FDCF.GOES-17.txt,...,292.378,294.698,293.493,77.944,43.385,-9999,22,3661,1134,2022-02-10 00:20:31+00:00
509992,509992,WFABBA / Fire Detection and Characterization A...,6_6_001g,2022-02-10 00:16:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2022041001617.rev.6_6_001g.FDCC.GOES-17.txt,...,292.654,295.230,293.878,77.174,43.385,-9999,22,3661,1134,2022-02-10 00:16:17+00:00


In [19]:
def matches(events_df, wfabba_df, minutes_delta, degree_delta):
    all_matches_df = pd.DataFrame()
    for row in events_df.iterrows():
        row = row[1]
        # print(row[1])
        time_lower = wfabba_df["timestamp_converted"] >= (row["datetime"] - timedelta(minutes=minutes_delta))
        time_upper = wfabba_df["timestamp_converted"] <= (row["datetime"] + timedelta(minutes=minutes_delta))

        # NOTE: 0.5 deg = 34.4861 miles
        lat_lower = wfabba_df["Latitude"] >= (row["lat_adjusted"] - degree_delta)
        lat_upper = wfabba_df["Latitude"] <= (row["lat_adjusted"] + degree_delta)
        long_lower = wfabba_df["Longitude"] >= (row["long_adjusted"] - degree_delta)
        long_upper = wfabba_df["Longitude"] <= (row["long_adjusted"] + degree_delta)
        print("FIgLib Event Datetime:",row["datetime"])
        print("FIgLib Event Latitude:",row["lat_adjusted"]) 
        print("FIgLib Event Longitude:",row["long_adjusted"])
        print("")
        print("WFABBA matches:")
        # match_results_df = wfabba_df[time_lower & time_upper & lat_lower & lat_upper & long_lower & long_upper][["Latitude", "Longitude", "Timestamp"]]
        match_results_df = wfabba_df[time_lower & time_upper & lat_lower & lat_upper & long_lower & long_upper].copy()
        match_results_df["Event Lat"] =  row["lat_adjusted"]
        match_results_df["Event Long"] = row["long_adjusted"]
        match_results_df["Event Datetime"] = row["datetime"]
        match_results_df["Lat Diff"] = (match_results_df["Latitude"] - match_results_df["Event Lat"]).abs()
        match_results_df["Long Diff"] = (match_results_df["Longitude"] - match_results_df["Event Long"]).abs()
        match_results_df["Time Diff"] = (match_results_df["timestamp_converted"] - match_results_df["Event Datetime"]).dt.total_seconds()/60
        match_results_df["Event Name"] = row["event_name"][:-1]
        # print("A:", match_results_df["timestamp_converted"], "B:", match_results_df["Event Datetime"], "Diff:", match_results_df["Time Diff"])
        print(match_results_df[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]])
        all_matches_df = pd.concat([all_matches_df, match_results_df])
        # print(all_matches_df)
        print("============================")
    return all_matches_df
    # break


In [20]:
# goes16_matches = matches(overlap_events_df, wfabba_goes_16_df, 60, 0.8)
goes16_matches = matches(overlap_events_df, wfabba_goes_16_df, 300, 0.5)
goes16_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
# goes16_matches[["Lat Diff", "Long Diff", "Time Diff"]].describe()
# goes16_matches[["Lat Diff", "Long Diff", "Time Diff"]]
goes16_matches
# goes16_matches[["Lat Diff", "Long Diff", "Time Diff"]].mean()

FIgLib Event Datetime: 2021-02-04 19:09:13+00:00
FIgLib Event Latitude: 33.5235
FIgLib Event Longitude: -116.4257

WFABBA matches:
Empty DataFrame
Columns: [Latitude, Longitude, timestamp_converted, Event Lat, Event Long, Event Datetime, Lat Diff, Long Diff, Time Diff]
Index: []
FIgLib Event Datetime: 2021-02-09 17:49:27+00:00
FIgLib Event Latitude: 33.363018
FIgLib Event Longitude: -116.83622

WFABBA matches:
        Latitude  Longitude       timestamp_converted  Event Lat  Event Long  \
145647   33.3402  -116.7544 2021-02-09 21:41:07+00:00  33.363018  -116.83622   
145654   33.3402  -116.7544 2021-02-09 21:46:07+00:00  33.363018  -116.83622   
145655   33.3402  -116.7544 2021-02-09 21:36:07+00:00  33.363018  -116.83622   
145659   33.3402  -116.7544 2021-02-09 21:40:10+00:00  33.363018  -116.83622   
145660   33.3402  -116.7544 2021-02-09 21:50:10+00:00  33.363018  -116.83622   

                  Event Datetime  Lat Diff  Long Diff   Time Diff  
145647 2021-02-09 17:49:27+00:00  0.0

,Unnamed: 0,Algorithm,Version,Timestamp,Satellite,Instrument,FlightModel,ScanMode,ProductType,FileName,...,FD_X,FD_Y,timestamp_converted,Event Lat,Event Long,Event Datetime,Lat Diff,Long Diff,Time Diff,Event Name
145647,145647,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:41:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214107.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:41:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,231.666667,20210209_FIRE_hp-e-mobo-c
145654,145654,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:46:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214607.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:46:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,236.666667,20210209_FIRE_hp-e-mobo-c
145655,145655,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:36:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040213607.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:36:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,226.666667,20210209_FIRE_hp-e-mobo-c
145659,145659,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:40:10,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214010.rev.6_6_001g.FDCF.GOES-16.txt,...,1063,1087,2021-02-09 21:40:10+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,230.716667,20210209_FIRE_hp-e-mobo-c
145660,145660,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:50:10,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040215010.rev.6_6_001g.FDCF.GOES-16.txt,...,1063,1087,2021-02-09 21:50:10+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,240.716667,20210209_FIRE_hp-e-mobo-c
145647,145647,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:41:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214107.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:41:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,203.866667,20210209_FIRE_tp-w-mobo-c
145654,145654,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:46:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214607.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:46:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,208.866667,20210209_FIRE_tp-w-mobo-c
145655,145655,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:36:07,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040213607.rev.6_6_001g.FDCC.GOES-16.txt,...,1063,1087,2021-02-09 21:36:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,198.866667,20210209_FIRE_tp-w-mobo-c
145659,145659,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:40:10,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040214010.rev.6_6_001g.FDCF.GOES-16.txt,...,1063,1087,2021-02-09 21:40:10+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,202.916667,20210209_FIRE_tp-w-mobo-c
145660,145660,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:50:10,GOES-16,ABI,FM?,GOES-16,GOES-16,f2021040215010.rev.6_6_001g.FDCF.GOES-16.txt,...,1063,1087,2021-02-09 21:50:10+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,212.916667,20210209_FIRE_tp-w-mobo-c


In [21]:
goes17_matches = matches(overlap_events_df, wfabba_goes_17_df, 300, 0.5)
goes17_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
# goes17_matches[["Lat Diff", "Long Diff", "Time Diff"]].describe()
# goes17_matches[["Lat Diff", "Long Diff", "Time Diff"]]
goes17_matches
# goes17_matches[["Lat Diff", "Long Diff", "Time Diff"]].mean()

FIgLib Event Datetime: 2021-02-04 19:09:13+00:00
FIgLib Event Latitude: 33.5235
FIgLib Event Longitude: -116.4257

WFABBA matches:
Empty DataFrame
Columns: [Latitude, Longitude, timestamp_converted, Event Lat, Event Long, Event Datetime, Lat Diff, Long Diff, Time Diff]
Index: []
FIgLib Event Datetime: 2021-02-09 17:49:27+00:00
FIgLib Event Latitude: 33.363018
FIgLib Event Longitude: -116.83622

WFABBA matches:
       Latitude  Longitude       timestamp_converted  Event Lat  Event Long  \
48867   33.3277  -116.7060 2021-02-09 21:36:17+00:00  33.363018  -116.83622   
48879   33.3452  -116.8964 2021-02-09 21:33:25+00:00  33.363018  -116.83622   
48893   33.7501  -117.0510 2021-02-09 22:30:25+00:00  33.363018  -116.83622   

                 Event Datetime  Lat Diff  Long Diff   Time Diff  
48867 2021-02-09 17:49:27+00:00  0.035318    0.13022  226.833333  
48879 2021-02-09 17:49:27+00:00  0.017818    0.06018  223.966667  
48893 2021-02-09 17:49:27+00:00  0.387082    0.21478  280.966667  
F

,Unnamed: 0,Algorithm,Version,Timestamp,Satellite,Instrument,FlightModel,ScanMode,ProductType,FileName,...,FD_X,FD_Y,timestamp_converted,Event Lat,Event Long,Event Datetime,Lat Diff,Long Diff,Time Diff,Event Name
48867,48867,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:36:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021040213617.rev.6_6_001g.FDCC.GOES-17.txt,...,3597,1043,2021-02-09 21:36:17+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.035318,0.13022,226.833333,20210209_FIRE_hp-e-mobo-c
48879,48879,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:33:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021040213325.rev.6_6_001g.FDCM1.GOES-17.txt,...,3589,1042,2021-02-09 21:33:25+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.017818,0.06018,223.966667,20210209_FIRE_hp-e-mobo-c
48893,48893,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 22:30:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021040223025.rev.6_6_001g.FDCM1.GOES-17.txt,...,3578,1025,2021-02-09 22:30:25+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.387082,0.21478,280.966667,20210209_FIRE_hp-e-mobo-c
48867,48867,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:36:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021040213617.rev.6_6_001g.FDCC.GOES-17.txt,...,3597,1043,2021-02-09 21:36:17+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.195800,0.28030,199.033333,20210209_FIRE_tp-w-mobo-c
48879,48879,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-02-09 21:33:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021040213325.rev.6_6_001g.FDCM1.GOES-17.txt,...,3589,1042,2021-02-09 21:33:25+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.178300,0.47070,196.166667,20210209_FIRE_tp-w-mobo-c
613665,613665,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-11 16:09:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021192160925.rev.6_6_001g.FDCM1.GOES-17.txt,...,3600,1056,2021-07-11 16:09:25+00:00,33.008700,-116.97090,2021-07-11 17:55:54+00:00,0.006700,0.24980,-106.483333,20210711_FIRE_wc-e-mobo-c
615925,615925,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-11 16:55:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021192165525.rev.6_6_001g.FDCM1.GOES-17.txt,...,3582,1040,2021-07-11 16:55:25+00:00,33.008700,-116.97090,2021-07-11 17:55:54+00:00,0.379100,0.08430,-60.483333,20210711_FIRE_wc-e-mobo-c
616125,616125,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-11 22:51:17,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021192225117.rev.6_6_001g.FDCC.GOES-17.txt,...,3606,1068,2021-07-11 22:51:17+00:00,33.008700,-116.97090,2021-07-11 17:55:54+00:00,0.277300,0.31600,295.383333,20210711_FIRE_wc-e-mobo-c
617309,617309,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-11 15:15:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021192151525.rev.6_6_001g.FDCM1.GOES-17.txt,...,3593,1045,2021-07-11 15:15:25+00:00,33.008700,-116.97090,2021-07-11 17:55:54+00:00,0.267000,0.15250,-160.483333,20210711_FIRE_wc-e-mobo-c
617701,617701,WFABBA / Fire Detection and Characterization A...,6_6_001g,2021-07-11 16:57:25,GOES-17,ABI,FM?,GOES-17,GOES-17,f2021192165725.rev.6_6_001g.FDCM1.GOES-17.txt,...,3581,1040,2021-07-11 16:57:25+00:00,33.008700,-116.97090,2021-07-11 17:55:54+00:00,0.378300,0.10890,-58.483333,20210711_FIRE_wc-e-mobo-c


In [22]:
# wfabba_goes_16_df[time_lower & time_upper & lat_lower & lat_upper & long_lower & long_upper][["Latitude", "Longitude", "Timestamp"]]

In [23]:
#Checking truth of FIgLib images
df_train = pd.read_json("train_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_train["type"] = "train"
df_train 

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20170519_FIRE_rm-w-mobo-c/1495223946_-02340,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
1,20170519_FIRE_rm-w-mobo-c/1495224006_-02280,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
2,20170519_FIRE_rm-w-mobo-c/1495224066_-02220,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
3,20170519_FIRE_rm-w-mobo-c/1495224126_-02160,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
4,20170519_FIRE_rm-w-mobo-c/1495224186_-02100,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
...,...,...,...,...,...,...,...
10433,20200521_FIRE_om-w-mobo-c/1590101441_+02160,20200521_FIRE_om-w-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
10434,20200521_FIRE_om-w-mobo-c/1590101501_+02220,20200521_FIRE_om-w-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
10435,20200521_FIRE_om-w-mobo-c/1590101561_+02280,20200521_FIRE_om-w-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
10436,20200521_FIRE_om-w-mobo-c/1590101621_+02340,20200521_FIRE_om-w-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train


In [24]:
df_test = pd.read_json("test_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_test["type"] = "test"
df_test 

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test
...,...,...,...,...,...,...,...
4880,20161113_FIRE_bm-n-mobo-c/1479069033_+02100,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4881,20161113_FIRE_bm-n-mobo-c/1479069093_+02160,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4882,20161113_FIRE_bm-n-mobo-c/1479069153_+02220,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4883,20161113_FIRE_bm-n-mobo-c/1479069213_+02280,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test


In [25]:
df_valid = pd.read_json("valid_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_valid["type"] = "valid"
df_valid

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
1,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
2,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
3,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
...,...,...,...,...,...,...,...
4906,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4907,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4908,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4909,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [26]:
df_labels = pd.concat([df_train, df_test, df_valid]).reset_index().drop(columns = ["index"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20170519_FIRE_rm-w-mobo-c/1495223946_-02340,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
1,20170519_FIRE_rm-w-mobo-c/1495224006_-02280,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
2,20170519_FIRE_rm-w-mobo-c/1495224066_-02220,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
3,20170519_FIRE_rm-w-mobo-c/1495224126_-02160,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
4,20170519_FIRE_rm-w-mobo-c/1495224186_-02100,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train
...,...,...,...,...,...,...,...
20229,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
20230,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
20231,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
20232,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [27]:
# get necessary values
df_labels["date"] = df_labels["camera_name"].str.split("_", n=1, expand=True)[0]
df_labels["year"] = df_labels["date"].str[:4]
df_labels["date"] = pd.to_datetime(df_labels["date"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20170519_FIRE_rm-w-mobo-c/1495223946_-02340,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,2017-05-19,2017
1,20170519_FIRE_rm-w-mobo-c/1495224006_-02280,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,2017-05-19,2017
2,20170519_FIRE_rm-w-mobo-c/1495224066_-02220,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,2017-05-19,2017
3,20170519_FIRE_rm-w-mobo-c/1495224126_-02160,20170519_FIRE_rm-w-mobo-c,0,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,2017-05-19,2017
4,20170519_FIRE_rm-w-mobo-c/1495224186_-02100,20170519_FIRE_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,2017-05-19,2017
...,...,...,...,...,...,...,...,...,...
20229,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
20230,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
20231,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
20232,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020


In [28]:
pd.DataFrame(df_labels[df_labels["date"] > "2021-01-20"][["camera_name","type", "date"]].groupby(["camera_name","type", "date"]).size().reset_index())

,camera_name,type,date,0
0,20210204_FIRE_tp-s-mobo-c,test,2021-02-04,80
1,20210209_FIRE_hp-e-mobo-c,test,2021-02-09,77
2,20210209_FIRE_tp-w-mobo-c,valid,2021-02-09,76
3,20210711_FIRE_wc-e-mobo-c,test,2021-07-11,80


In [29]:
df_labels_filtered = df_labels[df_labels["date"] > "2021-01-20"].reset_index().drop(columns=["index"])
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021
1,20210209_FIRE_hp-e-mobo-c/1612890687_-02280,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021
2,20210209_FIRE_hp-e-mobo-c/1612890747_-02220,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021
3,20210209_FIRE_hp-e-mobo-c/1612890807_-02160,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021
4,20210209_FIRE_hp-e-mobo-c/1612890867_-02100,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021
...,...,...,...,...,...,...,...,...,...
308,20210209_FIRE_tp-w-mobo-c/1612896795_+02160,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021
309,20210209_FIRE_tp-w-mobo-c/1612896855_+02220,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021
310,20210209_FIRE_tp-w-mobo-c/1612896915_+02280,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021
311,20210209_FIRE_tp-w-mobo-c/1612896975_+02340,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021


In [30]:
df_join = df_labels_filtered.merge(goes16_matches, left_on="camera_name", right_on="Event Name", how="left")
df_join

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,Unnamed: 0,...,FD_X,FD_Y,timestamp_converted,Event Lat,Event Long,Event Datetime,Lat Diff,Long Diff,Time Diff,Event Name
0,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021,145647.0,...,1063.0,1087.0,2021-02-09 21:41:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,231.666667,20210209_FIRE_hp-e-mobo-c
1,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021,145654.0,...,1063.0,1087.0,2021-02-09 21:46:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,236.666667,20210209_FIRE_hp-e-mobo-c
2,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021,145655.0,...,1063.0,1087.0,2021-02-09 21:36:07+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,226.666667,20210209_FIRE_hp-e-mobo-c
3,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021,145659.0,...,1063.0,1087.0,2021-02-09 21:40:10+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,230.716667,20210209_FIRE_hp-e-mobo-c
4,20210209_FIRE_hp-e-mobo-c/1612890627_-02340,20210209_FIRE_hp-e-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2021-02-09,2021,145660.0,...,1063.0,1087.0,2021-02-09 21:50:10+00:00,33.363018,-116.83622,2021-02-09 17:49:27+00:00,0.022818,0.08182,240.716667,20210209_FIRE_hp-e-mobo-c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,20210209_FIRE_tp-w-mobo-c/1612897035_+02400,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021,145647.0,...,1063.0,1087.0,2021-02-09 21:41:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,203.866667,20210209_FIRE_tp-w-mobo-c
1721,20210209_FIRE_tp-w-mobo-c/1612897035_+02400,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021,145654.0,...,1063.0,1087.0,2021-02-09 21:46:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,208.866667,20210209_FIRE_tp-w-mobo-c
1722,20210209_FIRE_tp-w-mobo-c/1612897035_+02400,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021,145655.0,...,1063.0,1087.0,2021-02-09 21:36:07+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,198.866667,20210209_FIRE_tp-w-mobo-c
1723,20210209_FIRE_tp-w-mobo-c/1612897035_+02400,20210209_FIRE_tp-w-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2021-02-09,2021,145659.0,...,1063.0,1087.0,2021-02-09 21:40:10+00:00,33.523500,-116.42570,2021-02-09 18:17:15+00:00,0.183300,0.32870,202.916667,20210209_FIRE_tp-w-mobo-c


In [31]:
coords_df = goes16_matches[goes16_matches["Event Name"]=="20210209_FIRE_hp-e-mobo-c"][["Latitude","Longitude"]].copy().reset_index().drop(columns=["index"])
coords_df.loc[len(coords_df.index)] = [33.363018,-116.83622]
coords_df

,Latitude,Longitude
0,33.340200,-116.75440
1,33.340200,-116.75440
2,33.340200,-116.75440
3,33.340200,-116.75440
4,33.340200,-116.75440
5,33.363018,-116.83622


In [32]:
# TODO - FIgLib Cleanup: 
# 1) Update the event names to use "_" instead of "-" as the separator
# ex: "FIRE-pi-s-mobo-c", "FIRE-smer-tcs8-mobo-c"
# 
# 2) 9 FIgLib events that lacked a pic with +00000.jpg. 
# ex: ['20170708_Whittier_syp-n-mobo-m/',
# '20180809_FIRE_bh-s-mobo-c/',
# '20190610_Pauma_bh-w-mobo-c/',
# '20190610_Pauma_bh-w-mobo-m/',
# '20200828_BorderFire_om-w-mobo-c/',
# '20200831_FIRE_wc-n-mobo-c/',
# '20200905_ValleyFire_cp-s-mobo-c/',
# '20201202_WillowFire-nightime-near-CDF-HQ_lp-w-mobo-c/',
# '20210110_Borderfire_lp-s-mobo-c/']